In [ ]:
# preprocessing with dask
import os, sys, re, io, pathlib
import pandas as pd
import hiplot as hip
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools

idx = pd.IndexSlice
buffer = io.StringIO()
idx = pd.IndexSlice

# define the current path (notebooks in lab_utils)
labutilspath = str(pathlib.Path(os.getcwd()).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

pp = autoscan.basics(material_info = True)

def hip_visualize(df, pcols = None, index = ['family', 'code']):
    dp = df.reset_index().loc[:, np.append(index, pcols)]
    s = hip.Experiment.from_dataframe(dp)
    s.colormap = 'interpolateViridis'
    s.display()
    return s

def ix_before_and_after(ds, index = ['tag', 'subtag'], columns = 'experiment', values = 'ix', mask = None, subset = None):
    ds.loc[:, 'ix'] = ds.index.values
    if mask is not None:
        ds = ds.loc[mask, :]
    dx = ds.pivot_table(index = index, 
                        columns = columns, 
                        values = values, 
                        aggfunc = lambda x: [*x])
    if subset is not None:
        dx = dx.loc[:, subset]
    dx.dropna(thresh = 2, inplace = True)

    # dp.loc[idx[dx.index[1], :], 'ix']

    # get the index of samples with before and after characterization 
    ix = list(itertools.chain(*dx.apply(lambda x: [*itertools.chain(*[s for s in x if type(s) != float])], axis = 1).values))
    return ix

def set_spe_style(ax, title = '', xlabel = '', ylabel=''):
    plt.sca(ax)
    plt.title(title, fontweight = 'bold');
    plt.xlabel(xlabel, fontweight = 'bold')
    plt.ylabel(ylabel, fontweight = 'bold');
    plt.xticks(fontweight = 'bold');
    plt.yticks(fontweight = 'bold');
    return ax

sns.set_context("paper", 
                rc = {
                    "fontsize":12,
                    'fontweight':'bold',
                    "axes.titlesize":12,
                    "axes.titleweight":'bold',
                    "axes.labelsize":12,
                    'axes.labelweight':'bold',
                    'xtick.labelsize':12,
                    'xtick.labelweight':'bold',
                    'ytick.labelsize':12,
                    'ytick.labelweight':'bold',
                    'legend.frameon':True,
                    'legend.fontsize':12,
                    'legend.title_fontsize':12,
                    'legend.fontweight':'bold',
                    'legeld.title_fontweight':'bold',
                    'title.fontsize':12
                })

sns.set_style('darkgrid')

In [ ]:
# define paths
datapath = '/sandbox/data/autoscan/'
vispath = '/sandbox/vis/autoscan/'
savepath = datapath
datafname = 'autoscan_corrected.h5'

datafile = os.path.join(datapath, datafname)
figspath = os.path.join(vispath, 'rock_multiphysics_display')

In [ ]:
# read the data
dd = pd.read_hdf(datafile, key = 'data')
ds = pd.read_hdf(datafile, key = 'desc')
df = ds.join(dd)

In [ ]:
for k,v in {'sandstone':1e3, "shale":1e2, 'carbonate':1e3}.items():
    ix = df.loc[(df.family == k) & (df.instance == 'before'), 'perm'] > v
    fill_val = df.loc[ix.index[ix == False], 'perm'].mean()
    df.loc[ix.index[ix == True], 'perm'] = fill_val

In [ ]:
df_rocks = df.query("family != 'metal' & family != 'gemstones'")
df_rocks = df_rocks.sort_values(by = 'instance', ascending = False)

In [ ]:
# define important columns (categorical and numerical)
col_numerical = pp.grid_cols + pp.probe_cols + ['l_max_peak', 'l_min_peak']

In [ ]:
# separate the values in another dataframe
ix = ix_before_and_after(ds, subset = ['before', 'heat_treatment', 'perf'])
df_bna = df.loc[ix, :]

In [ ]:
ix = ix_before_and_after(ds, mask = df.perm.isna() == False, subset = ['before', 'heat_treatment', 'perf'])
df_perm_bna = df.loc[ix, pp.meta_cols + pp.grid_cols + ['perm']]

In [ ]:
mask = df.loc[:, pp.mech_cols[::2]].isna().any(axis = 1) == False
ix = ix_before_and_after(ds, mask = mask.values, subset = ['before', 'heat_treatment'])
df_mech_bna = df.loc[ix, pp.meta_cols + pp.grid_cols + pp.mech_cols + pp.vel_cols]

In [ ]:
mask = df.loc[:, pp.ftir_cols].isna().any(axis = 1) == False
ix = ix_before_and_after(ds, mask = df.perm.isna() == False, subset = ['before', 'heat_treatment', 'perf'])
df_ftir_bna = df.loc[ix, pp.meta_cols + pp.grid_cols + pp.ftir_cols + ['l_max_peak']]

# visualization
1. hip-plot (again) but with corrected data
2. distributions

In [ ]:
# mech_cols = df.columns[df.columns.str.contains(r'mech[_][a-z]0')].values
# mech_cols = ['mech_' + s + '0' for s in ['e', 'k','n']]
subset_cols = ['l_max_peak', 'perm'] + pp.mech_cols[::2]

## hip
### without `e_star`

In [ ]:
df_before = df.query("instance == 'before'")
df_after  = df.query("instance == 'after'")

In [ ]:
s = hip_visualize(df_before.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_before_ftirmaxloc-perm-and-mechx0.html'));

In [ ]:
s = hip_visualize(df_after.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_after_ftirmaxloc-perm-and-mechx0.html'));

### with `e_star`
the number of samples with impulse hammer measurements are 1/4th of the previous

In [ ]:
subset_cols += ['e_star']

In [ ]:
s = hip_visualize(df_before.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_before_ftirmaxloc-perm-and-mechx0-estar.html'));

In [ ]:
s = hip_visualize(df_after.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_after_ftirmaxloc-perm-and-mechx0-estar.html'));

In [ ]:
subset_cols = subset_cols[1:-1]

In [ ]:
s = hip_visualize(df_before.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_before_permvel-mechx0.html'));

In [ ]:
s = hip_visualize(df_after.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_after_permvel-mechx0.html'));

In [ ]:
subset_cols = ['l_max_peak'] + subset_cols[1:]

In [ ]:
s = hip_visualize(df_before.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_before_ftirmaxloc-mechx0.html'));

In [ ]:
s = hip_visualize(df_after.dropna(subset = subset_cols), 
                  pcols = subset_cols, 
                  index = ['code'])

s.to_html(os.path.join(figspath, 'hip_after_ftirmaxloc-mechx0.html'));

In [ ]:
groupby_cols =['code', 'instance']
probes = ['perm']
subset_cols = groupby_cols + probes

In [ ]:
# # df_bna.loc[:, subset_cols].plot(kind = 'kde')
# fig, ax = plt.subplots(figsize = (12, 12))
# sns.kdeplot(x = 'perm',  hue = 'code', data = df_bna, clip = [0, 500], vertical = True,
#             palette = 'viridis', shade = 'fill', ax = ax)
# sns.set_style('darkgrid')
# plt.title('permeability before');

In [ ]:
ix_perm = df_before.perm.isna() == False
df_perm_before = df_before.loc[ix_perm, ['family', 'code', 'perm']]

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.stripplot(y = 'perm', x = 'family', hue = 'instance', dodge = True, data = df_rocks, 
              palette = 'viridis', ax = ax)
plt.yscale('log')
ax = set_spe_style(ax, title = 'Permeability before & after', xlabel = '', ylabel = 'Permeability (mD)')

In [ ]:
plt.xticks

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.boxplot(x = 'family', y = 'perm', hue = 'instance', data = df_rocks, dodge = True, width = 0.5)
plt.yscale('log')

In [ ]:
# fig, ax = plt.subplots(figsize = (12, 12))
# sns.stripplot(y = 'perm', x = 'family', hue = 'instance', dodge = True, data = df_perm_bna, palette = 'viridis', ax = ax)
# plt.yscale('log')
# sns.set_style('darkgrid')
# plt.title('permeability before & after');
# plt.xlabel('')
# plt.ylabel('Permeability (mD)')

In [ ]:
# fig, ax = plt.subplots(figsize = (12, 12))
# sns.boxplot(x = 'family', y = 'perm', hue = 'instance', data = df_perm_bna, dodge = True, width = 0.5)
# plt.yscale('log')

In [ ]:
df_perm_before_clipped = df_perm_before.copy()
df_perm_before_clipped.loc[:, 'perm'] = df_perm_before_clipped.perm.clip(lower = 0, upper = 500)

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.violinplot(y = 'perm', x = 'code', hue = 'family', data = df_perm_before_clipped, palette = 'viridis', ax = ax)
sns.set_style('darkgrid')
plt.title('permeability before');

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.boxplot(y = 'perm', x = 'code', hue = 'family', data = df_perm_before_clipped, palette = 'viridis', ax = ax)
sns.set_style('darkgrid')
plt.title('permeability before');

In [ ]:
fig, ax = plt.subplots(figsize = (12, 12))
sns.kdeplot(x = 'perm',  hue = 'code', data = df_perm_before_clipped, 
            palette = 'viridis', shade = 'fill', ax = ax)
sns.set_style('darkgrid')
plt.title('permeability before');

In [ ]:
tags = ds.tag.str.split('_', expand = True)#.apply(lambda x: pp.get_material_density(x))
tags[1] = 0.0
unique_tags = tags[0].unique()
# tags.set_index([0, tags.index], inplace = True)